In [1]:
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests
import pandas as pd

# Defining of the dataframe
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
df = pd.DataFrame(columns=['postcode'])
table = BeautifulSoup(requests.get('https://www.worldpostalcodes.org/l1/en/au/australia/list/r1/list-of-postcodes-in-victoria',headers = headers).text, "html.parser")
# Collecting Ddata
for row in table.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        postcode = columns[0].text.strip()

        df = df.append({'postcode':postcode}, ignore_index=True)
postcodes = df['postcode'].tolist()
postcodes = [x for x in postcodes if x]

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_7897/607283987.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcode':postcode}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_7897/607283987.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcode':postcode}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_7897/607283987.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcode':postcode}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_7897/607283987.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

In [2]:
postcodes

['3000',
 '3002',
 '3003',
 '3004',
 '3006',
 '3008',
 '3011',
 '3012',
 '3013',
 '3015',
 '3016',
 '3018',
 '3019',
 '3020',
 '3021',
 '3022',
 '3023',
 '3024',
 '3025',
 '3026',
 '3027',
 '3028',
 '3029',
 '3030',
 '3031',
 '3032',
 '3033',
 '3034',
 '3036',
 '3037',
 '3038',
 '3039',
 '3040',
 '3041',
 '3042',
 '3043',
 '3044',
 '3045',
 '3046',
 '3047',
 '3048',
 '3049',
 '3051',
 '3052',
 '3053',
 '3054',
 '3055',
 '3056',
 '3057',
 '3058',
 '3059',
 '3060',
 '3061',
 '3062',
 '3063',
 '3064',
 '3065',
 '3066',
 '3067',
 '3068',
 '3070',
 '3071',
 '3072',
 '3073',
 '3074',
 '3075',
 '3076',
 '3078',
 '3079',
 '3081',
 '3082',
 '3083',
 '3084',
 '3085',
 '3087',
 '3088',
 '3089',
 '3090',
 '3091',
 '3093',
 '3094',
 '3095',
 '3096',
 '3097',
 '3099',
 '3101',
 '3102',
 '3103',
 '3104',
 '3105',
 '3106',
 '3107',
 '3108',
 '3109',
 '3111',
 '3113',
 '3114',
 '3115',
 '3116',
 '3121',
 '3122',
 '3123',
 '3124',
 '3125',
 '3126',
 '3127',
 '3128',
 '3129',
 '3130',
 '3131',
 '3132',
 

In [3]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.
This is by no means production code.
"""
# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 31) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
for postcode in postcodes:
    postcode_url = BASE_URL + f"/rent/melbourne-region-vic/?postcode={postcode}&sort=price-desc&page="
    for page in N_PAGES:
        url = postcode_url + f"{page}"
        bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser")
        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.
        if bs_object is not None:
            index_links = bs_object \
                .find(
                    "ul",
                    {"data-testid": "results"}
                ) \
                .findAll(
                    "a",
                    href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
                )

            for link in index_links:
                # if its a property address, add it to the list
                if 'address' in link['class']:
                    url_links.append(link['href'])
        else:
            break

KeyboardInterrupt: 

In [ ]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url,headers = headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    
    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text
    
    property_metadata[property_url]['property_type'] = bs_object \
            .find("div", {"data-testid": "listing-summary-property-type"}) \
            .text
    '''
    property_metadata[property_url]['agent'] = [
        feature.text for feature in bs_object \
            .find("div", {"data-testid": "listing-details__residential-header-agent-copy"}) \
            .findAll("a")]
    '''
    
    for li_tag in bs_object.find_all('ul', {'data-testid':'listing-summary-strip'}):
        value=[]
        for span_tag in li_tag.find_all('li'):           
            value.append(span_tag.find('strong').text)
            property_metadata[property_url]['property_feature'] = value
    
    
    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]
    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    
    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')
    

# output to example json in data/raw/

with open('unfiltered.json', 'w') as f:
    dump(property_metadata, f)

In [ ]:
import pandas as pd
df_raw = pd.read_json('unfiltered.json')
df_raw = df_raw.T
df_raw = df_raw.reset_index(drop=True)
df_raw